# Learning to balance

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo, TimeLimit
from stable_baselines3 import PPO
import numpy as np
import os
from pathlib import Path
import control
import pybullet as p
import pybullet_data
import time
from IPython.display import clear_output, display
import pinocchio as pin
import pink

from pinocchio import RobotWrapper

In [ ]:
current = Path.cwd()

# If launched from a subfolder (VS Code), go one level up
if (current / "notebooks").exists():
    PROJECT_ROOT = current
else:
    PROJECT_ROOT = current.parent

os.chdir(PROJECT_ROOT)

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

## Set up the environment

In [ ]:
N_STEPS = 1500 # 30 seconds

In [ ]:
env = gym.make("CartPole-v1", max_episode_steps=N_STEPS)

In [ ]:
env_gui = gym.make("CartPole-v1", render_mode="rgb_array", max_episode_steps=N_STEPS)

env_gui = RecordVideo(
    env_gui,
    video_folder="videos",
    episode_trigger=lambda ep: True,
    name_prefix="balance_demo_test"
)

## Helpers

In [ ]:
def rollout_episode(env, model, max_steps=1500, deterministic=True):

    obs, _ = env.reset()

    states = []
    actions = []
    rewards = []

    for _ in range(max_steps):
        states.append(obs.copy())

        action, _ = model.predict(obs, deterministic=deterministic)
        actions.append(action)

        obs, reward, terminated, truncated, _ = env.step(action)
        rewards.append(reward)

        if terminated or truncated:
            break

    return np.array(states), np.array(actions), np.array(rewards)


In [ ]:
def rollout_episode_video(env, model, max_steps=1500, deterministic=True):
    obs, _ = env.reset()
    done = False
    steps = 0

    while not done and steps < max_steps:
        action, _ = model.predict(obs, deterministic=deterministic)
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        steps += 1


## Loading the data

In [ ]:
data = np.load("data/test_ppo_balance_clean_30s.npz")

obs      = data["observations"]
acts     = data["actions"]
ep_ids   = data["episode_ids"]
ep_lens  = data["episode_lens"]
dt       = float(data["dt"])

print(obs.shape, acts.shape, ep_lens)

Our dataset contains several episodes concatenated in time. However, for system identification we must only use transitions that stay inside the same episode, because the transition from the last state of one episode to the first state of the next episode is not physical (it comes from a reset).

To handle this safely, we use the array ep_ids, which tells us which episode each sample belongs to.

In [ ]:
FORCE_MAG = 10.0

same_ep = ep_ids[1:] == ep_ids[:-1]
X      = obs[:-1][same_ep]
X_next = obs[1:][same_ep]
a      = acts[:-1][same_ep]          # in {0, 1}

U_force = FORCE_MAG * (2 * a - 1)    # shape (N,)
U_force = U_force.reshape(-1, 1)     # (N, 1)

print("Transitions:", X.shape, U_force.shape, X_next.shape)

The linear model we want to identify is only valid **locally**, i.e., for small deviations around the upright equilibrium of the pendulum ($\theta \approx 0$). If we include samples where the pole has a large angle or spins quickly, the linear approximation becomes invalid and the estimated $A, B$ matrices will be inaccurate.

For this reason, we filter the dataset to keep only samples close to equilibrium.


In [ ]:
theta     = X[:, 2]
theta_dot = X[:, 3]
local_mask = (np.abs(theta) < 0.25) & (np.abs(theta_dot) < 1.0)

X_loc      = X[local_mask]
U_loc      = U_force[local_mask]
X_next_loc = X_next[local_mask]

print("Local samples:", X_loc.shape[0], "/", X.shape[0])

## Fitting the linear model

In [ ]:
nx = X_loc.shape[1]
nu = U_loc.shape[1]

Z = np.hstack([X_loc, U_loc])
Y = X_next_loc

W, *_ = np.linalg.lstsq(Z, Y, rcond=None)
A = W[:nx, :].T
B = W[nx:, :].T

print("A:\n", A)
print("B:\n", B)


## Testing the result

In [ ]:
Q = np.diag([1.0, 0.1, 10.0, 0.1])   # state penalty
R = np.array([[0.01]])             # control effort penalty

K, S, eigvals = control.dlqr(A, B, Q, R)

print("K =", K)
print("Closed-loop eigenvalues:", eigvals)

In [ ]:
FORCE_MAG = 10.0

def lqr_to_action(u):
    return 1 if u > 0 else 0

In [ ]:
class LQRModel:
    def __init__(self, K):
        self.K = K

    def predict(self, obs, deterministic=True):
        x = obs.reshape(-1, 1)
        u = -self.K @ x

        action = 1 if u.item() > 0 else 0

        return action, None


In [ ]:
lqr_model = LQRModel(K)

rollout_episode_video(
    env_gui,
    lqr_model,
    max_steps=1500,
    deterministic=True
)

env_gui.close()

**It works!**

## Initialize PyBullet

In [ ]:
import pybullet as p
import pybullet_data
import numpy as np
import time

# Import the Robot you want (e.g., ur3_description, panda_description...) 
from robot_descriptions import ur3_description

In [ ]:
p.connect(p.GUI)
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=45, cameraPitch=-30, cameraTargetPosition=[0.3, 0, 0])
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

# Select your Robot
robot_id = p.loadURDF(ur3_description.URDF_PATH, useFixedBase=True)

# Identify movable joints
num_joints = p.getNumJoints(robot_id)
joint_indices = [
    i for i in range(num_joints) 
    if p.getJointInfo(robot_id, i)[2] != p.JOINT_FIXED
]


print("\n Joints mapping:")
for i, joint_index in enumerate(joint_indices):
    joint_info = p.getJointInfo(robot_id, joint_index)
    joint_name = joint_info[1].decode("utf-8") 
    print(f"q[{i}] -> Index PyBullet {joint_index} : {joint_name}")

# Setup Robot for torque control
for joint_index in joint_indices:
    p.setJointMotorControl2(
        bodyUniqueId=robot_id,
        jointIndex=joint_index,
        controlMode=p.VELOCITY_CONTROL,
        force=0
    )

## Make the Robot go to the target configuration

In [ ]:
# Targets (position and velocity) 
q_target = np.array([0, -np.pi/2, 0, 0, 0, 0])
v_target = np.zeros(len(joint_indices))

# Define the PD gains
Kp = np.array([800.0, 800.0, 600.0, 50.0, 20.0, 20.0])
Kd = Kp*0.004


dt = 1./240.

try:
    while True:
        joint_states = p.getJointStates(robot_id, joint_indices)

        q_mes = np.array([state[0] for state in joint_states])
        v_mes = np.array([state[1] for state in joint_states])

        q_error = q_target - q_mes
        v_error = v_target - v_mes

        tau = Kp * q_error + Kd * v_error

        p.setJointMotorControlArray(
            bodyUniqueId=robot_id,
            jointIndices=joint_indices,
            controlMode=p.TORQUE_CONTROL,
            forces=tau
        )

        p.stepSimulation()
        time.sleep(dt)

except KeyboardInterrupt:
    p.disconnect()
    print("\nSimulation stopped")

## Closed-loop inverse kinematics with Pink

In [ ]:
import pybullet as p
import pybullet_data
import pinocchio as pin
import pink
from pink import solve_ik
from pink.tasks import FrameTask, PostureTask
from robot_descriptions.loaders.pinocchio import load_robot_description
from robot_descriptions import ur3_description
import numpy as np
import time


p.connect(p.GUI)
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=45, cameraPitch=-30, cameraTargetPosition=[0.3, 0, 0])
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

robot_id = p.loadURDF(ur3_description.URDF_PATH, useFixedBase=True)

num_joints = p.getNumJoints(robot_id)
joint_indices = [
    i for i in range(num_joints) 
    if p.getJointInfo(robot_id, i)[2] != p.JOINT_FIXED
]

for joint_index in joint_indices:
    p.setJointMotorControl2(robot_id, joint_index, p.VELOCITY_CONTROL, force=0)


## Setup pink 
# Load Pinocchio model of UR3
pin_robot = load_robot_description("ur3_description")
configuration = pink.Configuration(pin_robot.model, pin_robot.data, pin_robot.q0)


## IK task: control the pose (position + orientation) of the wrist frame
# Higher position_cost makes the solver prioritize position accuracy
end_effector_task = FrameTask("wrist_3_link", position_cost=10.0, orientation_cost=1.0)
tasks = [end_effector_task]

# Define a vertical line segment the end-effector will follow sinusoidally
y_start, y_end = -0.3, 0.3
x_pos, z_height = 0.2, 0.3
p.addUserDebugLine([x_pos, y_start, z_height], [x_pos, y_end, z_height], [1, 0, 0], lineWidth=3)

dt = 1./240.
t = 0.0

# PD gains
Kp = np.array([800.0, 800.0, 600.0, 50.0, 20.0, 20.0])
Kd = Kp*0.004


try:
    while True:
        # Compute moving target (oscillating along Y)
        y_target = y_start + (np.sin(t*3) + 1) / 2 * (y_end - y_start)
        
        target_rotation = pin.utils.rpyToMatrix(np.pi, 0, 0)
        target_translation = np.array([x_pos, y_target, z_height])
        
        end_effector_task.set_target(pin.SE3(target_rotation, target_translation))
        
        p.addUserDebugLine(target_translation, target_translation + [0,0,0.1], [0,1,0], lifeTime=dt*2)

        joint_states = p.getJointStates(robot_id, joint_indices)
        q_mes = np.array([state[0] for state in joint_states])
        v_mes = np.array([state[1] for state in joint_states])

        # Update Pink configuration with measured state
        configuration.update(q_mes)

        ## Solve IK using Pink (returns joint velocity command)
        velocity = solve_ik(configuration, tasks, dt, solver="quadprog")
        # Desired next configuration
        q_target = q_mes + velocity*dt
        v_target = velocity

        tau = Kp*(q_target - q_mes) + Kd*(v_target - v_mes)

        p.setJointMotorControlArray(
            bodyUniqueId=robot_id,
            jointIndices=joint_indices,
            controlMode=p.TORQUE_CONTROL,
            forces=tau
        )

        p.stepSimulation()
        time.sleep(dt)
        t += dt

except KeyboardInterrupt:
    p.disconnect()
    print("\nSimulation stopped")

## New file to extend the UR3 robot with a pendulum

In [1]:
import os
from robot_descriptions import ur3_description
import re

original_urdf_path = ur3_description.URDF_PATH
robot_root_path = os.path.dirname(os.path.dirname(original_urdf_path))


with open(original_urdf_path, 'r') as file:
    original_content = file.read()

pendulum_xml = """
  <joint name="pendulum_joint" type="revolute">
    <parent link="tool0"/>
    <child link="pendulum_link"/>
    <origin rpy="0 1.57 0" xyz="0 0 0"/>
    <axis xyz="0 0 1"/> 
    <limit effort="0" velocity="100" lower="-10.0" upper="10.0"/>
    <dynamics damping="0.01" friction="0.01"/>
  </joint>

  <link name="pendulum_link">
    <inertial>
      <origin rpy="0 0 0" xyz="0 0 0.29"/> 
      <mass value="0.5"/>
      <inertia ixx="0.001" ixy="0" ixz="0" iyy="0.001" iyz="0" izz="0.001"/>
    </inertial>
    
    <visual>
      <origin rpy="0 0 0" xyz="0 0 0.145"/>
      <geometry>
        <cylinder length="0.29" radius="0.01"/>
      </geometry>
      <material name="Gray">
        <color rgba="0.5 0.5 0.5 1"/>
      </material>
    </visual>

    <visual>
      <origin rpy="0 0 0" xyz="0 0 0.29"/>
      <geometry>
        <sphere radius="0.025"/>
      </geometry>
      <material name="Red">
        <color rgba="1 0 0 1"/>
      </material>
    </visual>
    
    <collision>
      <origin rpy="0 0 0" xyz="0 0 0.145"/>
      <geometry>
        <cylinder length="0.29" radius="0.01"/>
      </geometry>
    </collision>
  </link>
"""

problematic_prefix = "package://example-robot-data/robots/ur_description"
fixed_content = original_content.replace(problematic_prefix, robot_root_path)

final_content = fixed_content.replace('</robot>', pendulum_xml + '\n</robot>')

new_filename = "ur3_pendulum.urdf"
new_filepath = os.path.abspath(new_filename)

with open(new_filepath, 'w') as file:
    file.write(final_content)

In [2]:
import pybullet as p
import pybullet_data
import pinocchio as pin
import pink
from pink import solve_ik
from pink.tasks import FrameTask
import numpy as np
import time
import os

p.connect(p.GUI)
p.resetDebugVisualizerCamera(cameraDistance=1.2, cameraYaw=45, cameraPitch=-20, cameraTargetPosition=[0.3, 0, 0.3])
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

new_urdf_path = "ur3_pendulum.urdf"
robot_id = p.loadURDF(os.path.abspath(new_urdf_path), useFixedBase=True)

num_joints = p.getNumJoints(robot_id)
all_joint_indices = [i for i in range(num_joints) if p.getJointInfo(robot_id, i)[2] != p.JOINT_FIXED]

pendulum_joint_name = "pendulum_joint"
pendulum_index = -1
controlled_joints = []

print("Joints found:")
for i in all_joint_indices:
    info = p.getJointInfo(robot_id, i)
    name = info[1].decode('utf-8')
    print(f" - ID {i}: {name}")
    
    if name == pendulum_joint_name:
        pendulum_index = i
    else:
        controlled_joints.append(i)

p.setJointMotorControlArray(
    robot_id, 
    all_joint_indices, 
    p.VELOCITY_CONTROL, 
    forces=np.zeros(len(all_joint_indices))
)

pin_robot = pin.RobotWrapper.BuildFromURDF(
    new_urdf_path, 
    package_dirs=["."]
)

wrist_3_id = pin_robot.model.getJointId("wrist_3_joint")
idx_q = pin_robot.model.joints[wrist_3_id].idx_q
pin_robot.model.upperPositionLimit[idx_q] = 100.0
pin_robot.model.lowerPositionLimit[idx_q] = -100.0


q_init = np.array([0, -1.57, 0, -1.57, 0, 0, 0.0])
configuration = pink.Configuration(pin_robot.model, pin_robot.data, q_init)

end_effector_task = FrameTask("tool0", position_cost=10.0, orientation_cost=0.0)
tasks = [end_effector_task]

y_start, y_end = -0.0, 0.0
x_pos, z_height = 0.45, 0.4
p.addUserDebugLine([x_pos, y_start, z_height], [x_pos, y_end, z_height], [1, 0, 0], lineWidth=3)

dt = 1./240.
t = 0.0

Kp = np.array([800.0, 800.0, 600.0, 50.0, 20.0, 10.0]) * 0.6
Kd = Kp * 0.005

for i, val in zip(controlled_joints, q_init[:-1]):
    p.resetJointState(robot_id, i, val)

if pendulum_index != -1:
    p.resetJointState(robot_id, pendulum_index, q_init[-1])

try:
    while True:
        y_target = y_start + (np.sin(t*1) + 1) / 2 * (y_end - y_start)
        
        target_rotation = np.eye(3) 
        target_translation = np.array([x_pos, y_target, z_height])
        end_effector_task.set_target(pin.SE3(target_rotation, target_translation))
        
        joint_states = p.getJointStates(robot_id, all_joint_indices)
        q_mes_all = np.array([state[0] for state in joint_states])
        v_mes_all = np.array([state[1] for state in joint_states])
        
        # Update Pink
        configuration.update(q_mes_all)
        
        # Solve IK
        velocity_all = solve_ik(configuration, tasks, dt, solver="quadprog", safety_break=False)
        
        # Split Arm / Pendulum
        q_mes_arm = q_mes_all[:-1] 
        v_mes_arm = v_mes_all[:-1]
        vel_cmd_arm = velocity_all[:-1] 

        # PD Controller        
        q_target_arm = q_mes_arm + vel_cmd_arm * dt

        pin.computeGeneralizedGravity(pin_robot.model, pin_robot.data, q_mes_all)
        g = pin_robot.data.g
        
        tau_arm = Kp * (q_target_arm - q_mes_arm) \
                    + Kd * (vel_cmd_arm - v_mes_arm) + g[:-1]
        
        # Pendulum is passive
        tau_pendulum = [0.0]

        p.setJointMotorControlArray(robot_id, controlled_joints, p.TORQUE_CONTROL, forces=tau_arm)
        p.setJointMotorControlArray(robot_id, [pendulum_index], p.TORQUE_CONTROL, forces=tau_pendulum)
        

        p.stepSimulation()
        time.sleep(dt)
        t += dt

except KeyboardInterrupt:
    p.disconnect()
    print("\nSimulation stopped")

pybullet build time: Dec  4 2025 20:11:42


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Iris(R) Xe Graphics (RPL-U)
GL_VERSION=4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.1
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.1
Vendor = Intel
Renderer = Mesa Intel(R) Iris(R) Xe Graphics (RPL-U)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubunt

: 